In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import json
import gzip
import ast

# Predict Average rating of the places

In [ ]:
reviews = pd.read_csv("/content/drive/Shareddrives/RS/LA02/Reviews/Reviews_preprocessed_positive.csv", lineterminator='\n' )

In [ ]:
reviews.head()

,Unnamed: 0,rating,reviewerName,reviewText,categories,gPlusPlaceId,gPlusUserId,sentiment
0,1,4.0,william spindler,Best War Wanton soup in Red Bluff,"['Asian Restaurant', 'Chinese Restaurant']",106591714648856494903,100000032416892623125,positive
1,2,5.0,william spindler,This is a review that is long overdo. I've bee...,"['European Restaurant', 'Italian Restaurant', ...",109420033090810328045,100000032416892623125,positive
2,3,5.0,william spindler,"Some authentic rub BBQ, great food and don't m...",['Barbecue Restaurant'],111623070919810985923,100000032416892623125,positive
3,4,4.0,william spindler,Truly a Red Bluff standard. Great old fashione...,['Restaurant'],113854191152597312098,100000032416892623125,positive
4,5,5.0,william spindler,"Long time favorite Mexican food, always consis...",['Mexican Restaurant'],115827996910815192564,100000032416892623125,positive


In [ ]:
reviews.shape

(4337645, 8)

In [ ]:
import gzip
import pandas as pd
# import goslate
import json
import requests

import nltk
import string
from nltk.stem.porter import *
from nltk.corpus import stopwords

nltk.download("stopwords")
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from collections import defaultdict
from datetime import datetime
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
final_place_df = pd.read_csv("/content/drive/Shareddrives/RS/LA02/Places/Final_places.csv", lineterminator='\n' )

In [ ]:
final_place_df.head()

,Unnamed: 0,Unnamed: 0.1,name,address,hours,phone,closed,gPlusPlaceId,gps
0,0,1500000,Pep Boys Auto Parts & Service,"['7625 N Academy Blvd', 'Colorado Spgs, CO 809...","[['Monday', [['8:00 am--8:00 pm']]], ['Tuesday...",(719) 599-4455,False,106604455956312584856,"[38.94162, -104.797586]"
1,1,1500001,Acclaim Hearing Center,"['110', '7321 W Charleston Blvd', 'Las Vegas, ...","[['Monday', [['9:00 am--5:00 pm']]], ['Tuesday...",(702) 363-2336,False,108865044857061260547,"[36.159274, -115.278781]"
2,2,1500005,Zooland Park,"['Route de Cazaux', '33260 La Teste-de-Buch', ...","[['Monday', [['10:00 am--7:00 pm']]], ['Tuesda...",05 56 54 71 44,False,102023846774373264702,"[44.585871, -1.136379]"
3,3,1500006,BlueChip Software GmbH,"['Hochtennstraße 30', '5700 Zell am See', 'Aus...","[['Monday', [['8:00 am--12:00 pm'], ['1:00--5:...",06542 575840,False,110328490284324713981,"[47297029.0, 12798235.0]"
4,4,1500014,Cupolens Gatukök & Kiosk,"['Majgatan 1', '582 46 Linköping', 'Sweden']","[['Monday', [['11:00 am--10:00 pm']]], ['Tuesd...",013-10 35 37,False,116680537008710602604,"[58.407777, 15.590062]"


In [ ]:
final_place_df.shape

(1341688, 9)

##Function - Filtering out data based on GPS coordinates

In [ ]:
import re
# Define the extraction functions
# Define the extraction functions with error handling
def extract_lat(s):
    pattern = r"[-]?\d+\.\d+"
    match = re.findall(pattern, s)
    if match:
        return float(match[0])
    else:
        return None

def extract_lng(s):
    pattern = r"[-]?\d+\.\d+"
    match = re.findall(pattern, s)
    if match:
        return float(match[1])
    else:
        return None


# Apply the extraction functions to the column and save to new columns
final_place_df['lat'] = final_place_df['gps'].apply(extract_lat)
final_place_df['lng'] = final_place_df['gps'].apply(extract_lng)

final_place_df['gps']

0           [38.94162, -104.797586]
1          [36.159274, -115.278781]
2            [44.585871, -1.136379]
3          [47297029.0, 12798235.0]
4            [58.407777, 15.590062]
                     ...           
1341683      [25.05034, 121.370568]
1341684    [36.055789, -115.234458]
1341685    [33.837921, -118.378552]
1341686      [49.895228, 14.492974]
1341687     [338.18679, -786.73214]
Name: gps, Length: 1341688, dtype: object

In [ ]:
final_place_df['lat']

0          3.894162e+01
1          3.615927e+01
2          4.458587e+01
3          4.729703e+07
4          5.840778e+01
               ...     
1341683    2.505034e+01
1341684    3.605579e+01
1341685    3.383792e+01
1341686    4.989523e+01
1341687    3.381868e+02
Name: lat, Length: 1341688, dtype: float64

In [ ]:
final_place_df['lng']

0         -1.047976e+02
1         -1.152788e+02
2         -1.136379e+00
3          1.279824e+07
4          1.559006e+01
               ...     
1341683    1.213706e+02
1341684   -1.152345e+02
1341685   -1.183786e+02
1341686    1.449297e+01
1341687   -7.867321e+02
Name: lng, Length: 1341688, dtype: float64

In [ ]:
final_place_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'name', 'address', 'hours', 'phone',
       'closed', 'gPlusPlaceId', 'gps', 'lat', 'lng'],
      dtype='object')

In [ ]:
def places_filter(final_place_df, lat_l, lat_h, lon_l, lon_h):
    reviews_user = []
    count=0
    places_ca=final_place_df.loc[(final_place_df['lat'] >= lat_l)
              & (final_place_df['lat'] <= lat_h)
              & (final_place_df['lng'] >= lon_l)
              & (final_place_df['lng'] <= lon_h)]
    return places_ca

In [ ]:
places_ca =places_filter(final_place_df, 32.32, 42, -124.26, -114.8)

In [ ]:
places_ca.describe()

,Unnamed: 0,Unnamed: 0.1,lat,lng
count,119684.000000,1.196840e+05,119684.000000,119684.000000
mean,334582.763293,2.303053e+06,35.520124,-119.205034
std,193512.485853,4.656988e+05,2.063523,2.229363
min,1.000000,1.500001e+06,32.338862,-124.257755
25%,166449.000000,1.897308e+06,33.889290,-121.770964
50%,334375.500000,2.301208e+06,34.245379,-118.378492
75%,502064.000000,2.706173e+06,37.634637,-117.663602
max,670841.000000,3.114332e+06,41.948573,-114.828768


In [ ]:
len(places_ca)

119684

Merging review df and user df (Inner join on User Id)

In [ ]:
place_review_df = pd.merge(places_ca, reviews, on='gPlusPlaceId', how='inner')

In [ ]:
place_review_df.shape

(374094, 18)

In [ ]:
place_review_df[:1]

,Unnamed: 0_x,Unnamed: 0.1,name,address,hours,phone,closed,gPlusPlaceId,gps,lat,lng,Unnamed: 0_y,rating,reviewerName,reviewText,categories,gPlusUserId,sentiment
0,1,1500001,Acclaim Hearing Center,"['110', '7321 W Charleston Blvd', 'Las Vegas, ...","[['Monday', [['9:00 am--5:00 pm']]], ['Tuesday...",(702) 363-2336,False,108865044857061260547,"[36.159274, -115.278781]",36.159274,-115.278781,110321,5.0,Roger McManus,Dr. Otto took great care with my father's case...,['Audiologist'],100323516990085092116,positive


In [ ]:
place_review_df=place_review_df.drop(['Unnamed: 0_x', 'Unnamed: 0.1', 'Unnamed: 0_y'], axis=1)

In [ ]:
place_review_df.columns

Index(['name', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps',
       'lat', 'lng', 'rating', 'reviewerName', 'reviewText', 'categories',
       'gPlusUserId', 'sentiment'],
      dtype='object')

In [ ]:
categ_list = []
placeCateg = defaultdict(list)
count = 0
for d in place_review_df['categories']:
  if d not in categ_list:
    categ_list.append(d)

len(categ_list),  # categ_list

(15172,)

In [ ]:
categ_list[:2]

["['Audiologist']", "['Cell Phone Store', 'Electronics Store']"]

In [ ]:
import ast

string_list = categ_list
list_2d = [ast.literal_eval(string) for string in string_list]
list_1d = [item for sublist in list_2d for item in sublist]
print(list_1d)

['Audiologist', 'Cell Phone Store', 'Electronics Store', 'Lingerie Store', 'Ready Mix Concrete Supplier', 'Acupuncture Clinic', 'Karate School', 'Physical Fitness Program', 'Martial Arts Supply Store', 'European Restaurant', 'Italian Restaurant', 'Chiropractor', 'Massage Therapist', 'Medical Transcription Service', 'Latin American Restaurant', 'Mexican Restaurant', 'Tours', 'Restaurants', 'Kickboxing School', 'Asian Restaurant', 'Takeout Restaurant', 'Lebanese Restaurant', 'Bridal Shop', 'Jewelry Designer', 'Wedding Store', 'House Cleaning Service', 'Janitorial Service', 'Chiropractor', 'Sushi Restaurant', 'Asian Restaurant', 'Japanese Restaurant', 'History Museum', 'Sporting Goods Store', 'Entertainment and Recreation', 'Fast Food Restaurant', 'Latin American Restaurant', 'Mexican Restaurant', 'Copy Shop', 'Mailbox Rental Service', 'Mailing Service', 'Locksmith', 'Towing Service', 'Cosmetic Dentist', 'Comic Book Store', 'Game Store', 'Toy Store', 'Attorney', 'Employment Agency', 'Bank

In [ ]:
len(list_1d)

38464

In [ ]:
from collections import Counter

count=Counter(list_1d)
print(dict(count))

{'Audiologist': 7, 'Cell Phone Store': 80, 'Electronics Store': 97, 'Lingerie Store': 15, 'Ready Mix Concrete Supplier': 7, 'Acupuncture Clinic': 44, 'Karate School': 24, 'Physical Fitness Program': 96, 'Martial Arts Supply Store': 12, 'European Restaurant': 350, 'Italian Restaurant': 247, 'Chiropractor': 101, 'Massage Therapist': 107, 'Medical Transcription Service': 3, 'Latin American Restaurant': 199, 'Mexican Restaurant': 201, 'Tours': 19, 'Restaurants': 1, 'Kickboxing School': 14, 'Asian Restaurant': 374, 'Takeout Restaurant': 137, 'Lebanese Restaurant': 15, 'Bridal Shop': 35, 'Jewelry Designer': 28, 'Wedding Store': 19, 'House Cleaning Service': 27, 'Janitorial Service': 21, 'Sushi Restaurant': 67, 'Japanese Restaurant': 109, 'History Museum': 17, 'Sporting Goods Store': 72, 'Entertainment and Recreation': 66, 'Fast Food Restaurant': 339, 'Copy Shop': 52, 'Mailbox Rental Service': 48, 'Mailing Service': 68, 'Locksmith': 15, 'Towing Service': 38, 'Cosmetic Dentist': 98, 'Comic Boo

In [ ]:
sorted_dict = dict(sorted(count.items(), key=lambda item: item[1], reverse=True))

In [ ]:
sorted_dict

{'American Restaurant': 812,
 'Restaurant': 567,
 'Asian Restaurant': 374,
 'European Restaurant': 350,
 'Fast Food Restaurant': 339,
 'Bar': 330,
 'Cafe': 320,
 'Auto Repair Shop': 298,
 'Coffee Shop': 277,
 'Bakery': 275,
 'Sandwich Shop': 272,
 'Bar & Grill': 259,
 'Seafood Restaurant': 252,
 'Italian Restaurant': 247,
 'Used Car Dealer': 224,
 'Auto Parts Store': 222,
 'Deli': 217,
 'Department Store': 210,
 'Mexican Restaurant': 201,
 'Latin American Restaurant': 199,
 'Breakfast Restaurant': 195,
 'Brake Shop': 193,
 'Pizza Restaurant': 185,
 'Gift Shop': 178,
 'Dessert Shop': 173,
 'Cocktail Bar': 168,
 'Sports Bar': 164,
 'Mediterranean Restaurant': 163,
 'Wine Bar': 161,
 'Tire Shop': 150,
 'Steak House': 150,
 'Chinese Restaurant': 149,
 'Hamburger Restaurant': 148,
 'Caterer': 143,
 'Auto Body Shop': 142,
 'Lounge': 141,
 'Takeout Restaurant': 137,
 'Grocery Store': 136,
 'Day Spa': 136,
 'Catering': 126,
 'Beauty Salon': 125,
 'Auto Tune Up Service': 123,
 'Non-Profit Organ

In [ ]:
categ_list1=set(list_1d)
print(len(categ_list1))

2299


In [ ]:
categ_list1

{'Tour Operator',
 'Homeless Service',
 'Book Dealers-Retail',
 'Screen Printer',
 'Nail Salon',
 'Food Delivery',
 'Indian Reservation',
 'Car Sharing Location',
 'Soba Noodle Shop',
 'Group Home',
 'Musical Instrument Repair Shop',
 'Machine Shop',
 'Auto Glass Replacement',
 'Wheel Store',
 'Archive',
 'Gardening',
 'Fire Protection Service',
 'Self Service Car Wash',
 'Auto Repair',
 'Pet Care',
 'Country Club',
 'Coffee Houses & Cafes',
 'Baby Clothing Store',
 'Airport Parking Service',
 'Home Insurance Agency',
 'Roller Skating Rink',
 'Travel Clinic',
 'Dance Hall',
 'Water Damage Restoration...',
 'Truck Rental Agency',
 'Massage Spa',
 'Sushi Restaurant',
 'Security Guard Service',
 'Hair Salon',
 'Family Law Attorney',
 'Heaters-Unit-Dealers (whls)',
 'Bartending School',
 'Learning Center',
 'Auto Sunroof Shop',
 'Cemetery',
 'Public Swimming Pool',
 'Fishing Pier',
 'Volleyball Club',
 'Restaurants - Caterers',
 'Plastic Surgeon',
 'Australian Restaurant',
 'Food Bank',
 '

In [ ]:
perCategReviews = defaultdict(int)
for c in categ_list1:
    for d in list(place_review_df["categories"]):
            if c in d:
                perCategReviews[c] += 1

In [ ]:
perCategReviews

defaultdict(int,
            {'Tour Operator': 26,
             'Homeless Service': 12,
             'Book Dealers-Retail': 8,
             'Screen Printer': 154,
             'Nail Salon': 1334,
             'Food Delivery': 10,
             'Indian Reservation': 6,
             'Car Sharing Location': 14,
             'Soba Noodle Shop': 12,
             'Group Home': 2,
             'Musical Instrument Repair Shop': 96,
             'Machine Shop': 262,
             'Auto Glass Replacement': 4,
             'Wheel Store': 364,
             'Archive': 8,
             'Gardening': 2,
             'Fire Protection Service': 16,
             'Self Service Car Wash': 30,
             'Auto Repair': 8118,
             'Pet Care': 42,
             'Country Club': 86,
             'Coffee Houses & Cafes': 90,
             'Baby Clothing Store': 52,
             'Airport Parking Service': 2,
             'Home Insurance Agency': 430,
             'Roller Skating Rink': 22,
             'Trav

In [ ]:
top_categ = []
for d in perCategReviews:
    item = (perCategReviews[d], d)
    top_categ.append(item)

sorted(top_categ, reverse=True)[:10]

[(159158, 'Restaurant'),
 (50100, 'American Restaurant'),
 (44696, 'Store'),
 (35748, 'Service'),
 (29846, 'Asian'),
 (29336, 'Asian Restaurant'),
 (27526, 'Bar'),
 (20156, 'European Restaurant'),
 (19006, 'Mexican Restaurant'),
 (18028, 'Food')]